# **HTIN5005 - Assignment 2 (Hospital Resource Allocation)**

Alfian Nurfaizi (SID 550180999) <br>
Master of Digital Health and Data Science <br>

SCI-XAI Pipeline adapted from https://github.com/petmoreno/SCI-XAI-Pipeline.git

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Change Directory
%cd /content/drive/MyDrive/HTIN5005

/content/drive/MyDrive/HTIN5005


# **Import Libraries**

In [3]:
#Import general libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Import libraries for building the pipeline and join their branches
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

#Import SCI-XAI modules created for data preparation phase
import my_utils
import missing_val_imput
import feature_select
import preprocessing
import adhoc_transf

#Import libraries for data preparation phase
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder


#Import libraries from modelling phase
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

#Import Ensemble Trees Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier,VotingClassifier
import xgboost as xgb

#To save model fit with and avoid longer waits
import joblib
import os

# **Import Dataset**

In [4]:
#Loading the dataset
path_data=r'/content/drive/MyDrive/HTIN5005/Dataset.csv'

df=pd.read_csv(path_data)
df.head()

,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,BT,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Error_group,Length of stay_min,KTAS duration_min,mistriage
0,2,2,71,3,3,2,right ocular pain,1,1,2.0,...,36.6,100.0,2,Corneal abrasion,1,4,2,86,5.00,1
1,1,1,56,12,3,2,right forearm burn,1,1,2.0,...,36.5,NaN,4,"Burn of hand, firts degree dorsum",1,5,4,64,3.95,1
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2.0,...,36.6,98.0,4,"Fracture of surgical neck of humerus, closed",2,5,4,862,1.00,1
3,1,2,71,8,1,1,ascites tapping,1,1,3.0,...,36.5,NaN,4,Alcoholic liver cirrhosis with ascites,1,5,6,108,9.83,1
4,1,2,58,4,3,1,"distension, abd",1,1,3.0,...,36.5,NaN,4,Ascites,1,5,8,109,6.60,1


In [5]:
#Drop unused features
df=df.drop(['Patients number per hour','Error_group','KTAS duration_min', 'mistriage'],axis=1)

In [6]:
#Define the mapping
mapping = {
    'Group': {1: 'Local ED', 2: 'Regional ED'},
    'Sex': {1: 'Female', 2: 'Male'},
    'Arrival mode': {1: 'Walking', 2: '119 use', 3: 'Private car', 4: 'Private ambulance', 5: 'Public transportation', 6: 'Wheelchair', 7: 'Others'},
    'Injury': {1: 'Non-injury', 2: 'Injury'},
    'Mental': {1: 'Alert', 2: 'Verbal response', 3: 'Pain response', 4: 'Unconciousness'},
    'Pain': {1: 'Pain', 2: 'Non-pain'},
    'Disposition': {1: 'Discharge', 2: 'Ward admission', 3: 'ICU admission', 4: 'AMA discharge', 5: 'Transfer', 6: 'Death', 7: 'OP fom ED'}
}

# Replace numerical values with categorical labels
df.replace(mapping, inplace=True)

# Display the first few rows to show the changes
display(df.head())

,Group,Sex,Age,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,SBP,DBP,HR,RR,BT,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Length of stay_min
0,Regional ED,Male,71,Private car,Injury,right ocular pain,Alert,Pain,2.0,160.0,100.0,84.0,18.0,36.6,100.0,2,Corneal abrasion,Discharge,4,86
1,Local ED,Female,56,Private car,Injury,right forearm burn,Alert,Pain,2.0,137.0,75.0,60.0,20.0,36.5,NaN,4,"Burn of hand, firts degree dorsum",Discharge,5,64
2,Regional ED,Female,68,119 use,Injury,"arm pain, Lt",Alert,Pain,2.0,130.0,80.0,102.0,20.0,36.6,98.0,4,"Fracture of surgical neck of humerus, closed",Ward admission,5,862
3,Local ED,Male,71,Walking,Non-injury,ascites tapping,Alert,Pain,3.0,139.0,94.0,88.0,20.0,36.5,NaN,4,Alcoholic liver cirrhosis with ascites,Discharge,5,108
4,Local ED,Male,58,Private car,Non-injury,"distension, abd",Alert,Pain,3.0,91.0,67.0,93.0,18.0,36.5,NaN,4,Ascites,Discharge,5,109


In [7]:
#Characterising the dataset
target_feature='Disposition'
numerical_feats=['Age', 'NRS_pain', 'SBP', 'DBP', 'HR', 'RR', 'BT', 'Saturation']
nominal_feats=['Group', 'Sex', 'Arrival mode', 'Injury', 'Pain', 'Chief_complain', 'Diagnosis in ED']
ordinal_feats=['Mental', 'KTAS_RN', 'KTAS_expert']

len_numerical_feats=len(numerical_feats)
len_nominal_feats=len(nominal_feats)
len_ordinal_feats=len(ordinal_feats)

# **Exploratory Data Analysis**

In [8]:
#Statistical analysis
df.describe()

,Age,NRS_pain,SBP,DBP,HR,RR,BT,Saturation,KTAS_RN,KTAS_expert,Length of stay_min
count,1267.000000,711.000000,1242.000000,1238.000000,1247.000000,1245.000000,1249.000000,570.000000,1267.000000,1267.000000,1267.000000
mean,54.423836,4.104079,133.648953,79.780291,83.963111,19.506827,36.580624,97.024561,3.335438,3.265983,11016.102605
std,19.725033,1.419332,27.275639,15.154292,16.644096,2.016649,0.545708,4.350556,0.885391,0.885803,80446.092065
min,16.000000,1.000000,50.000000,31.000000,32.000000,14.000000,35.000000,20.000000,1.000000,1.000000,0.000000
25%,37.000000,3.000000,114.000000,70.000000,72.000000,18.000000,36.200000,97.000000,3.000000,3.000000,133.000000
50%,57.000000,4.000000,130.000000,80.000000,82.000000,20.000000,36.500000,98.000000,3.000000,3.000000,274.000000
75%,71.000000,5.000000,150.000000,90.000000,96.000000,20.000000,36.800000,98.000000,4.000000,4.000000,606.500000
max,96.000000,10.000000,275.000000,160.000000,148.000000,30.000000,41.000000,100.000000,5.000000,5.000000,709510.000000


In [9]:
#Identifying missing values
my_utils.info_adhoc(df)

,% non-null values,non-null values,dtype
Group,100.000000,1267,object
Sex,100.000000,1267,object
Age,100.000000,1267,int64
Arrival mode,100.000000,1267,object
Injury,100.000000,1267,object
Chief_complain,100.000000,1267,object
Mental,100.000000,1267,object
Pain,100.000000,1267,object
NRS_pain,56.116811,711,float64
SBP,98.026835,1242,float64


In [10]:
#Exploring unique characters
my_utils.df_values(df)

*****start of feature  Group *************************
Group
Local ED       688
Regional ED    579
Name: count, dtype: int64
*****end of feature  Group ************************** 

*****start of feature  Sex *************************
Sex
Male      661
Female    606
Name: count, dtype: int64
*****end of feature  Sex ************************** 

*****start of feature  Age *************************
Age
58    37
74    29
68    28
77    28
56    27
      ..
90     1
94     1
93     1
95     1
96     1
Name: count, Length: 81, dtype: int64
*****end of feature  Age ************************** 

*****start of feature  Arrival mode *************************
Arrival mode
Private car              753
119 use                  266
Private ambulance        155
Walking                   79
Wheelchair                10
Public transportation      2
Others                     2
Name: count, dtype: int64
*****end of feature  Arrival mode ************************** 

*****start of feature  Injury *********

# **Data Preprocessing**

In [11]:
#Remove 'NRS_pain' and 'Saturation' feature because it has a rate of non-values under 70%
df=df.drop(['NRS_pain', 'Saturation'],axis=1)

#Update the numerical, nominal and ordinal feats after removing feature
numerical_feats=['Age', 'SBP', 'DBP', 'HR', 'RR', 'BT']
len_numerical_feats=len(numerical_feats)

In [12]:
#Performing numeric cast for numerical features
df.loc[:,numerical_feats]=adhoc_transf.Numeric_Cast_Column().fit_transform(df.loc[:,numerical_feats])
# Removed explicit conversion to integer type as it caused issues with float values.
# The pipeline will handle imputation and scaling for numerical features.

df[numerical_feats].dtypes


>>>>>>>>Calling init() from Numeric_Cast_Column

>>>>>>>>Calling fit() from Numeric_Cast_Column

>>>>>>>>Calling transform() from Numeric_Cast_Column


,0
Age,int64
SBP,float64
DBP,float64
HR,float64
RR,float64
BT,float64


In [13]:
#Performing category cast for nominal features
df.loc[:,nominal_feats]=adhoc_transf.Category_Cast_Column().fit_transform(df.loc[:,nominal_feats])
df[nominal_feats].dtypes


>>>>>>>>Calling init() from Category_Cast_Column

>>>>>>>>Calling fit() from Category_Cast_Column

>>>>>>>>Calling transform() from Category_Cast_Column


,0
Group,object
Sex,object
Arrival mode,object
Injury,object
Pain,object
Chief_complain,object
Diagnosis in ED,object


In [14]:
#Performing category cast for ordinal features
#Convert all ordinal columns to string type before applying the custom transformer
for col in ordinal_feats:
    df[col] = df[col].astype(str)

df.loc[:,ordinal_feats]=adhoc_transf.Category_Cast_Column().fit_transform(df.loc[:,ordinal_feats])
df[ordinal_feats].dtypes


>>>>>>>>Calling init() from Category_Cast_Column

>>>>>>>>Calling fit() from Category_Cast_Column

>>>>>>>>Calling transform() from Category_Cast_Column


,0
Mental,object
KTAS_RN,object
KTAS_expert,object


In [15]:
#Transform multiclass variables with only top ten most frequent categories
columns_to_transform = ['Chief_complain', 'Diagnosis in ED']
num_top_categories = 10

for col in columns_to_transform:
    # Get the top 10 most frequent categories
    top_categories = df[col].value_counts().nlargest(num_top_categories).index.tolist()

    # Replace categories not in the top 10 with 'others'
    df[col] = df[col].apply(lambda x: x if x in top_categories else 'Others')

print("Transformation complete for:", columns_to_transform)

Transformation complete for: ['Chief_complain', 'Diagnosis in ED']


In [16]:
#Exploring wrong characters
my_utils.df_values(df)

*****start of feature  Group *************************
Group
Local ED       688
Regional ED    579
Name: count, dtype: int64
*****end of feature  Group ************************** 

*****start of feature  Sex *************************
Sex
Male      661
Female    606
Name: count, dtype: int64
*****end of feature  Sex ************************** 

*****start of feature  Age *************************
Age
58    37
74    29
68    28
77    28
56    27
      ..
90     1
94     1
93     1
95     1
96     1
Name: count, Length: 81, dtype: int64
*****end of feature  Age ************************** 

*****start of feature  Arrival mode *************************
Arrival mode
Private car              753
119 use                  266
Private ambulance        155
Walking                   79
Wheelchair                10
Public transportation      2
Others                     2
Name: count, dtype: int64
*****end of feature  Arrival mode ************************** 

*****start of feature  Injury *********

# **Train-Test Splitting**

In [17]:
# Final check on dtypes before splitting
print("\nFinal dtypes before splitting:")
print(df.dtypes)
print(f"\nTarget variable '{target_feature}' dtype: {df[target_feature].dtype}")


Final dtypes before splitting:
Group                  object
Sex                    object
Age                     int64
Arrival mode           object
Injury                 object
Chief_complain         object
Mental                 object
Pain                   object
SBP                   float64
DBP                   float64
HR                    float64
RR                    float64
BT                    float64
KTAS_RN                object
Diagnosis in ED        object
Disposition            object
KTAS_expert            object
Length of stay_min      int64
dtype: object

Target variable 'Disposition' dtype: object


In [18]:
#Split the dataset into train and test
test_ratio_split=0.3
#Ensure target variable is suitable for stratification (e.g., integer or string)
#Convert target to string temporarily for stratification if it's numeric but treated as categorical
if pd.api.types.is_numeric_dtype(df[target_feature]):
    stratify_target = df[target_feature].astype(str)
else:
    stratify_target = df[target_feature]


train_set,test_set=train_test_split(df, test_size=test_ratio_split, random_state=42, stratify=stratify_target)

X_train=train_set.drop(target_feature,axis=1)
y_train=train_set[target_feature].copy() # Keep original type for now

X_test=test_set.drop(target_feature,axis=1)
y_test=test_set[target_feature].copy() # Keep original type for now

print(f"Training set shape: X={X_train.shape}, y={y_train.shape}")
print(f"Testing set shape: X={X_test.shape}, y={y_test.shape}")
print("\nTraining set target distribution:")
print(y_train.value_counts(normalize=True))
print("\nTesting set target distribution:")
print(y_test.value_counts(normalize=True))

Training set shape: X=(886, 17), y=(886,)
Testing set shape: X=(381, 17), y=(381,)

Training set target distribution:
Disposition
Discharge         0.628668
Ward admission    0.294582
Transfer          0.024831
AMA discharge     0.020316
OP fom ED         0.018059
ICU admission     0.006772
Death             0.006772
Name: proportion, dtype: float64

Testing set target distribution:
Disposition
Discharge         0.629921
Ward admission    0.293963
Transfer          0.026247
AMA discharge     0.020997
OP fom ED         0.015748
Death             0.007874
ICU admission     0.005249
Name: proportion, dtype: float64


# **Label Encoding for Target Value**

In [19]:
print("\nApplying LabelEncoder to target variable...")
le=LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

print("Original classes:", le.classes_)
print("Encoded training labels (first 10):", y_train_encoded[:10])
print("Encoded testing labels (first 10):", y_test_encoded[:10])
# Use encoded labels from now on
y_train = y_train_encoded
y_test = y_test_encoded


Applying LabelEncoder to target variable...
Original classes: ['AMA discharge' 'Death' 'Discharge' 'ICU admission' 'OP fom ED'
 'Transfer' 'Ward admission']
Encoded training labels (first 10): [2 2 2 6 2 2 2 6 2 2]
Encoded testing labels (first 10): [6 2 2 3 2 2 6 2 2 2]


# **Build Data Preparation Pipeline**

In [20]:
print("\nBuilding data preparation pipelines...")
#Before a data preprocessing will take place for each type of feature
pipeline_numeric_feat=Pipeline([ ('data_missing',missing_val_imput.Numeric_Imputer(strategy='median')),
                                 ('scaler', MinMaxScaler())])

pipeline_nominal_feat=Pipeline([('data_missing',missing_val_imput.Category_Imputer(strategy='most_frequent')), # Use most_frequent for nominal
                                 ('encoding', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])#We dont use OneHotEncoder since it enlarges the number of nominal features

pipeline_ordinal_feat=Pipeline([ ('data_missing',missing_val_imput.Category_Imputer(strategy='most_frequent')),
                                 ('encoding', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])


Building data preparation pipelines...

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Numeric_Imputer

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Category_Imputer

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Category_Imputer


### **Pipeline Model**
Lets define 3 pipeline model: <br>
a) Parallel approach where feature selection is performed in parallel for numerical, nominal, and categorical. <br>
b) General approach where feature selection is performed as a whole for other features. <br>
c) No feature selection is performed.

In [21]:
#Option A
print("Defining Pipeline Option A (Parallel Feature Selection)...")
pipe_numeric_featsel=Pipeline([('data_prep',pipeline_numeric_feat),
                                ('feat_sel',feature_select.Feature_Selector(strategy='wrapper_RFE', k_out_features=len_numerical_feats//2 or 1) )]) # Select half or at least 1
pipe_nominal_featsel=Pipeline([('data_prep',pipeline_nominal_feat),
                                ('feat_sel',feature_select.Feature_Selector(strategy='wrapper_RFE', k_out_features=len_nominal_feats//2 or 1) )]) # Select half or at least 1
pipe_ordinal_featsel=Pipeline([('data_prep',pipeline_ordinal_feat),
                                ('feat_sel',feature_select.Feature_Selector(strategy='wrapper_RFE', k_out_features=len_ordinal_feats//2 or 1) )]) # Select half or at least 1

dataprep_pipe_opta=ColumnTransformer([('numeric_pipe',pipe_numeric_featsel,numerical_feats),
                                    ('nominal_pipe',pipe_nominal_featsel,nominal_feats),
                                    ('ordinal_pipe',pipe_ordinal_featsel,ordinal_feats)
                                ], remainder='passthrough')

Defining Pipeline Option A (Parallel Feature Selection)...

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Feature_Selector

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Feature_Selector

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Feature_Selector


In [22]:
#Option B
print("Defining Pipeline Option B (General Feature Selection)...")
dataprep_merge_feat_b=ColumnTransformer([('numeric_pipe',pipeline_numeric_feat,numerical_feats),
                                    ('nominal_pipe',pipeline_nominal_feat, nominal_feats),
                                    ('ordinal_pipe',pipeline_ordinal_feat,ordinal_feats)
                                ], remainder='passthrough')
# Select half of total features or at least 1
k_features_b = (len_numerical_feats + len_nominal_feats + len_ordinal_feats) // 2 or 1
dataprep_pipe_optb=Pipeline([('data_prep',dataprep_merge_feat_b),
                                ('feat_sel',feature_select.Feature_Selector(strategy='wrapper_RFE', k_out_features=k_features_b) )])

Defining Pipeline Option B (General Feature Selection)...

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling init() from Feature_Selector


In [23]:
#Option C
print("Defining Pipeline Option C (No Feature Selection)...")
dataprep_pipe_optc = ColumnTransformer([
    ('numeric_pipe', pipeline_numeric_feat, numerical_feats),
    ('nominal_pipe', pipeline_nominal_feat, nominal_feats),
    ('ordinal_pipe', pipeline_ordinal_feat, ordinal_feats)
], remainder='passthrough')

Defining Pipeline Option C (No Feature Selection)...


In [24]:
#Test fitting one pipe from Option A to see if it works
print("\nTesting fit of a sub-pipeline (pipe_nominal_featsel)...")
try:
    # Use a small subset for quick testing if needed
    # pipe_nominal_featsel.fit_transform(X_train[nominal_feats].head(10), y_train[:10])
    pipe_nominal_featsel.fit_transform(X_train[nominal_feats], y_train) # Fit on full training data
    print("Test fit of pipe_nominal_featsel successful.")
except Exception as e:
    print(f"Error fitting pipe_nominal_featsel: {e}")
    print("Check your custom modules (missing_val_imput, feature_select) or data types.")


Testing fit of a sub-pipeline (pipe_nominal_featsel)...

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling fit() from Category_Imputer

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:27 2025 >>>>>>>>Calling fit() from Feature_Selector

 Sun Oct 26 07:23:27 2025 ********Inside fit() from Feature_Selector - self.feat_sel:  RFE(estimator=LogisticRegression(max_iter=2000), n_features_to_select=3)

 Sun Oct 26 07:23:28 2025 >>>>>>>>Calling transform() from Feature_Selector
Test fit of pipe_nominal_featsel successful.


# **Classifier Initialisation**

In [25]:
print("\nInitialising classifiers...")
#Several ensemble classifier with Cross validation will be applied
#We take decision tree as base classifier

dectree_clf=DecisionTreeClassifier(random_state=42)
rndforest_clf=RandomForestClassifier(random_state=42)
extratree_clf=ExtraTreesClassifier(random_state=42)
ada_clf= AdaBoostClassifier(random_state=42)
# Specify objective for multiclass explicitly for xgboost
xgboost_clf= xgb.XGBClassifier(objective='multi:softprob', num_class=len(le.classes_), random_state=42, use_label_encoder=False, eval_metric='mlogloss')
gradboost_clf=GradientBoostingClassifier(random_state=42)
voting_clf=VotingClassifier(estimators=[('rdf', rndforest_clf), ('xtra', extratree_clf), ('ada', ada_clf)], voting='soft') # Soft voting needs predict_proba


Initialising classifiers...


# **Scoring Initialisation**

In [26]:
print("Initialising scoring metrics for multiclass...")
# Define kwargs for roc_auc_score separately
roc_auc_kwargs = {'average': 'weighted', 'multi_class': 'ovr'}
# Use 'weighted' average for metrics sensitive to class imbalance
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision_weighted': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall_weighted': make_scorer(recall_score, average='weighted', zero_division=0),
    'f1_weighted': make_scorer(f1_score, average='weighted', zero_division=0),
    'roc_auc_ovr_weighted': make_scorer(roc_auc_score, response_method='predict_proba', **roc_auc_kwargs),
    'mcc': make_scorer(matthews_corrcoef)
}

Initialising scoring metrics for multiclass...


In [27]:
# Define the refit metric
refit_metric = 'roc_auc_ovr_weighted'

# **Training Data with RandomizedSearchCV**

In [28]:
#Define number of iterations for RandomizedSearchCV
N_ITER_SEARCH = 50
print(f"\nRunning RandomizedSearchCV with n_iter={N_ITER_SEARCH}")


Running RandomizedSearchCV with n_iter=50


## **Model A. Parallel Approach**

In [29]:
print("\n--- Training Model A (Parallel Feature Selection) ---")
full_parallel_pipe_opta=Pipeline([('data_prep',dataprep_pipe_opta),('clf',DecisionTreeClassifier(random_state=42))]) # Start with a default clf
print("Pipeline A keys:")
print(full_parallel_pipe_opta.get_params().keys())


--- Training Model A (Parallel Feature Selection) ---
Pipeline A keys:
dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'data_prep', 'clf', 'data_prep__force_int_remainder_cols', 'data_prep__n_jobs', 'data_prep__remainder', 'data_prep__sparse_threshold', 'data_prep__transformer_weights', 'data_prep__transformers', 'data_prep__verbose', 'data_prep__verbose_feature_names_out', 'data_prep__numeric_pipe', 'data_prep__nominal_pipe', 'data_prep__ordinal_pipe', 'data_prep__numeric_pipe__memory', 'data_prep__numeric_pipe__steps', 'data_prep__numeric_pipe__transform_input', 'data_prep__numeric_pipe__verbose', 'data_prep__numeric_pipe__data_prep', 'data_prep__numeric_pipe__feat_sel', 'data_prep__numeric_pipe__data_prep__memory', 'data_prep__numeric_pipe__data_prep__steps', 'data_prep__numeric_pipe__data_prep__transform_input', 'data_prep__numeric_pipe__data_prep__verbose', 'data_prep__numeric_pipe__data_prep__data_missing', 'data_prep__numeric_pipe__data_prep__scaler', 'data_prep__nu

In [30]:
#Load or save the model saved to avoid a new fitting
path_model_a = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/clf_fpipe_a_random_allocation.pkl'
path_results_train_a = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_a_random_allocation.xlsx'
path_results_test_a = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_a_random_allocation.xlsx'

In [31]:
try:
    clf_fpipe_a= joblib.load(path_model_a)
    print("Loaded model 'a' from disk.")
    model_a_loaded = True
except FileNotFoundError:
    print(f"Model file 'a' not found at {path_model_a}. Fitting new model.")
    clf_fpipe_a = None
    model_a_loaded = False
except Exception as e:
    print(f"Error loading model 'a': {e}. Fitting new model.")
    clf_fpipe_a = None
    model_a_loaded = False

Loaded model 'a' from disk.


In [32]:
# RandomizedSearchCV samples from these lists.
param_grid_fpipe_a={'clf':[dectree_clf, rndforest_clf, extratree_clf, ada_clf, xgboost_clf, gradboost_clf],
                    'data_prep__numeric_pipe__data_prep__data_missing__strategy':['mean','median'],
                    'data_prep__numeric_pipe__feat_sel__k_out_features':[*range(1,len_numerical_feats+1)],
                    'data_prep__numeric_pipe__feat_sel__strategy':['filter_mutinf','wrapper_RFE'], # Removed filter_num as filter_mutinf is generally better
                    'data_prep__nominal_pipe__feat_sel__k_out_features':[*range(1,len_nominal_feats+1)],
                    'data_prep__nominal_pipe__feat_sel__strategy':['filter_mutinf','wrapper_RFE'], # Removed filter_cat
                    'data_prep__ordinal_pipe__feat_sel__k_out_features':[*range(1,len_ordinal_feats+1)],
                    'data_prep__ordinal_pipe__feat_sel__strategy':['filter_mutinf','wrapper_RFE'] # Removed filter_cat
                    }

In [33]:
if not model_a_loaded or not hasattr(clf_fpipe_a, 'best_estimator_'):
    if model_a_loaded: # Loaded but not fitted?
         print("Model 'a' loaded but seems unfit. Refitting.")
    clf_fpipe_a=RandomizedSearchCV(full_parallel_pipe_opta,
                                 param_distributions=param_grid_fpipe_a,
                                 n_iter=N_ITER_SEARCH,
                                 scoring=scoring,
                                 refit=refit_metric,
                                 cv=5, # Using 5-fold CV
                                 n_jobs=-1, # Use all available cores
                                 random_state=42,
                                 error_score='raise', # Raise errors to debug
                                 verbose=1) # Show progress
    print("Fitting model 'a'...")
    try:
        clf_fpipe_a.fit(X_train, y_train) # Use encoded y_train
        print("Fit complete for model 'a'.")
        #Saving the model
        try:
            os.makedirs(os.path.dirname(path_model_a), exist_ok=True) # Ensure directory exists
            joblib.dump(clf_fpipe_a, path_model_a, compress=1)
            print(f"Model 'a' saved to {path_model_a}")
        except Exception as e:
            print(f"Error saving model 'a': {e}")
    except Exception as e:
         print(f"ERROR during RandomizedSearchCV fit for Model A: {e}")
         import traceback
         traceback.print_exc()

else:
    print("Skipping fit for model 'a' as it was loaded from disk and appears to be fitted.")

Skipping fit for model 'a' as it was loaded from disk and appears to be fitted.


In [34]:
#Printing the best estimator for model A
if hasattr(clf_fpipe_a, 'best_params_'):
    print('\nBest estimator of clf_fpipe_a:', clf_fpipe_a.best_params_)
    print(f'Score ({refit_metric}) of best estimator of clf_fpipe_a:', clf_fpipe_a.best_score_)
    print('Best index',clf_fpipe_a.best_index_ )
else:
    print("Model 'a' has not been fitted successfully.")


Best estimator of clf_fpipe_a: {'data_prep__ordinal_pipe__feat_sel__strategy': 'filter_mutinf', 'data_prep__ordinal_pipe__feat_sel__k_out_features': 3, 'data_prep__numeric_pipe__feat_sel__strategy': 'filter_mutinf', 'data_prep__numeric_pipe__feat_sel__k_out_features': 3, 'data_prep__numeric_pipe__data_prep__data_missing__strategy': 'median', 'data_prep__nominal_pipe__feat_sel__strategy': 'filter_mutinf', 'data_prep__nominal_pipe__feat_sel__k_out_features': 6, 'clf': RandomForestClassifier(random_state=42)}
Score (roc_auc_ovr_weighted) of best estimator of clf_fpipe_a: 0.8409019290876099
Best index 45


In [35]:
#Saving the training results into dataframe
if hasattr(clf_fpipe_a, 'cv_results_'):
    df_results_clf_fpipe_a=pd.DataFrame(clf_fpipe_a.cv_results_).sort_values(f'rank_test_{refit_metric}')
    try:
        os.makedirs(os.path.dirname(path_results_train_a), exist_ok=True)
        df_results_clf_fpipe_a.to_excel(path_results_train_a,index=False)
        print(f"Training results 'a' saved to {path_results_train_a}")
    except Exception as e:
        print(f"Error saving training results 'a': {e}")
else:
    print("No CV results to save for model 'a'.")

Training results 'a' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_a_random_allocation.xlsx


In [36]:
#Performing test phase with test set for model A
if hasattr(clf_fpipe_a, 'best_estimator_'):
    print("\nEvaluating Model A on test set...")
    # clf_fpipe_a.refit # refit=True does this automatically
    y_pred_clf_fpipe_a=clf_fpipe_a.predict(X_test)
    y_proba_clf_fpipe_a = clf_fpipe_a.predict_proba(X_test) if hasattr(clf_fpipe_a.best_estimator_, 'predict_proba') else None


    acc_a = accuracy_score(y_test, y_pred_clf_fpipe_a)
    f1_a = f1_score(y_test, y_pred_clf_fpipe_a, average='weighted', zero_division=0)
    prec_a = precision_score(y_test, y_pred_clf_fpipe_a, average='weighted', zero_division=0)
    rec_a = recall_score(y_test, y_pred_clf_fpipe_a, average='weighted', zero_division=0)
    mcc_a = matthews_corrcoef(y_test, y_pred_clf_fpipe_a)
    # Correctly call roc_auc_score for multiclass with probabilities
    roc_auc_a = roc_auc_score(y_test, y_proba_clf_fpipe_a, average='weighted', multi_class='ovr') if y_proba_clf_fpipe_a is not None else None

    test_results_clf_fpipe_a={'clf':['clf_fpipe_a_multi_random'],
                     'params':[clf_fpipe_a.best_params_],
                     'accuracy_test':[acc_a],
                     'f1_weighted_test':[f1_a],
                     'precision_weighted_test':[prec_a],
                     'recall_weighted_test':[rec_a],
                     'mcc_test': [mcc_a],
                     'roc_auc_ovr_weighted_test':[roc_auc_a]
        }

    test_results_y_pred_clf_fpipe_a=pd.DataFrame(data=test_results_clf_fpipe_a)
    try:
        os.makedirs(os.path.dirname(path_results_test_a), exist_ok=True) # Ensure directory exists
        test_results_y_pred_clf_fpipe_a.to_excel(path_results_test_a,index=False)
        print(f"Test results 'a' saved to {path_results_test_a}")
    except Exception as e:
        print(f"Error saving test results 'a': {e}")

    print(f"Accuracy of test set (model a): {acc_a:.4f}")
    print(f"F1 (weighted) of test set (model a): {f1_a:.4f}")
    print(f"ROC AUC (OVR weighted) of test set (model a): {roc_auc_a:.4f}" if roc_auc_a is not None else "ROC AUC: N/A (predict_proba not available)")

else:
    print("Model 'a' was not fit successfully. Skipping test phase.")


Evaluating Model A on test set...

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:30 2025 >>>>>>>>Calling transform() from Feature_Selector


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instanc

Test results 'a' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_a_random_allocation.xlsx
Accuracy of test set (model a): 0.7165
F1 (weighted) of test set (model a): 0.6902
ROC AUC (OVR weighted) of test set (model a): 0.8324


## **Model B. General Approach**

In [37]:
print("\n--- Training Model B (General Feature Selection) ---")
full_parallel_pipe_optb=Pipeline([('data_prep',dataprep_pipe_optb),('clf',DecisionTreeClassifier(random_state=42))]) # Start with default clf
print("Pipeline B keys:")
print(full_parallel_pipe_optb.get_params().keys())


--- Training Model B (General Feature Selection) ---
Pipeline B keys:
dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'data_prep', 'clf', 'data_prep__memory', 'data_prep__steps', 'data_prep__transform_input', 'data_prep__verbose', 'data_prep__data_prep', 'data_prep__feat_sel', 'data_prep__data_prep__force_int_remainder_cols', 'data_prep__data_prep__n_jobs', 'data_prep__data_prep__remainder', 'data_prep__data_prep__sparse_threshold', 'data_prep__data_prep__transformer_weights', 'data_prep__data_prep__transformers', 'data_prep__data_prep__verbose', 'data_prep__data_prep__verbose_feature_names_out', 'data_prep__data_prep__numeric_pipe', 'data_prep__data_prep__nominal_pipe', 'data_prep__data_prep__ordinal_pipe', 'data_prep__data_prep__numeric_pipe__memory', 'data_prep__data_prep__numeric_pipe__steps', 'data_prep__data_prep__numeric_pipe__transform_input', 'data_prep__data_prep__numeric_pipe__verbose', 'data_prep__data_prep__numeric_pipe__data_missing', 'data_prep__data_prep__n

In [38]:
#Load the model saved to avoid a new fitting
path_model_b = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/clf_fpipe_b_multi_random_allocation.pkl'
path_results_train_b = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_b_multi_random_allocation.xlsx'
path_results_test_b = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_b_multi_random_allocation.xlsx'

try:
    clf_fpipe_b= joblib.load(path_model_b)
    print("Loaded model 'b' from disk.")
    model_b_loaded = True
except FileNotFoundError:
    print(f"Model file 'b' not found at {path_model_b}. Fitting new model.")
    clf_fpipe_b = None
    model_b_loaded = False
except Exception as e:
    print(f"Error loading model 'b': {e}. Fitting new model.")
    clf_fpipe_b = None
    model_b_loaded = False

Loaded model 'b' from disk.


In [39]:
param_grid_fpipe_b={'clf':[dectree_clf, rndforest_clf, extratree_clf, ada_clf, xgboost_clf, gradboost_clf],
                    'data_prep__data_prep__numeric_pipe__data_missing__strategy':['mean','median'],
                    'data_prep__feat_sel__k_out_features':[*range(1,len_numerical_feats+len_nominal_feats+len_ordinal_feats+1)],
                    'data_prep__feat_sel__strategy':['filter_mutinf','wrapper_RFE']
                    }

In [40]:
if not model_b_loaded or not hasattr(clf_fpipe_b, 'best_estimator_'):
    if model_b_loaded:
        print("Model 'b' loaded but seems unfit. Refitting.")
    clf_fpipe_b=RandomizedSearchCV(full_parallel_pipe_optb,
                                 param_distributions=param_grid_fpipe_b,
                                 n_iter=N_ITER_SEARCH,
                                 scoring=scoring,
                                 refit=refit_metric,
                                 cv=5,
                                 n_jobs=-1,
                                 random_state=42,
                                 error_score='raise',
                                 verbose=1)
    print("Fitting model 'b'...")
    try:
        clf_fpipe_b.fit(X_train, y_train) # Use encoded y_train
        print("Fit complete for model 'b'.")
        #Saving the model
        try:
            os.makedirs(os.path.dirname(path_model_b), exist_ok=True)
            joblib.dump(clf_fpipe_b, path_model_b, compress=1)
            print(f"Model 'b' saved to {path_model_b}")
        except Exception as e:
            print(f"Error saving model 'b': {e}")
    except Exception as e:
         print(f"ERROR during RandomizedSearchCV fit for Model B: {e}")
         import traceback
         traceback.print_exc()
else:
    print("Skipping fit for model 'b' as it was loaded from disk.")

Skipping fit for model 'b' as it was loaded from disk.


In [41]:
#Printing the best estimator for model B
if hasattr(clf_fpipe_b, 'best_params_'):
    print('\nBest estimator of clf_fpipe_b:', clf_fpipe_b.best_params_)
    print(f'Score ({refit_metric}) of best estimator of clf_fpipe_b:', clf_fpipe_b.best_score_)
    print('Best index',clf_fpipe_b.best_index_ )
else:
    print("Model 'b' has not been fitted successfully.")


Best estimator of clf_fpipe_b: {'data_prep__feat_sel__strategy': 'wrapper_RFE', 'data_prep__feat_sel__k_out_features': 15, 'data_prep__data_prep__numeric_pipe__data_missing__strategy': 'mean', 'clf': RandomForestClassifier(random_state=42)}
Score (roc_auc_ovr_weighted) of best estimator of clf_fpipe_b: 0.7294398118238806
Best index 23


In [42]:
#Saving the training results into dataframe
if hasattr(clf_fpipe_b, 'cv_results_'):
    df_results_clf_fpipe_b=pd.DataFrame(clf_fpipe_b.cv_results_).sort_values(f'rank_test_{refit_metric}')
    try:
        os.makedirs(os.path.dirname(path_results_train_b), exist_ok=True)
        df_results_clf_fpipe_b.to_excel(path_results_train_b,index=False)
        print(f"Training results 'b' saved to {path_results_train_b}")
    except Exception as e:
        print(f"Error saving training results 'b': {e}")
else:
    print("No CV results to save for model 'b'.")

Training results 'b' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_b_multi_random_allocation.xlsx


In [43]:
#Performing test phase with test set for model B
if hasattr(clf_fpipe_b, 'best_estimator_'):
    print("\nEvaluating Model B on test set...")
    # clf_fpipe_b.refit
    y_pred_clf_fpipe_b=clf_fpipe_b.predict(X_test)
    y_proba_clf_fpipe_b = clf_fpipe_b.predict_proba(X_test) if hasattr(clf_fpipe_b.best_estimator_, 'predict_proba') else None

    acc_b = accuracy_score(y_test, y_pred_clf_fpipe_b)
    f1_b = f1_score(y_test, y_pred_clf_fpipe_b, average='weighted', zero_division=0)
    prec_b = precision_score(y_test, y_pred_clf_fpipe_b, average='weighted', zero_division=0)
    rec_b = recall_score(y_test, y_pred_clf_fpipe_b, average='weighted', zero_division=0)
    mcc_b = matthews_corrcoef(y_test, y_pred_clf_fpipe_b)
    # Correctly call roc_auc_score for multiclass with probabilities
    roc_auc_b = roc_auc_score(y_test, y_proba_clf_fpipe_b, average='weighted', multi_class='ovr') if y_proba_clf_fpipe_b is not None else None


    test_results_clf_fpipe_b={'clf':['clf_fpipe_b_multi_random'],
                     'params':[clf_fpipe_b.best_params_],
                     'accuracy_test':[acc_b],
                     'f1_weighted_test':[f1_b],
                     'precision_weighted_test':[prec_b],
                     'recall_weighted_test':[rec_b],
                     'mcc_test': [mcc_b],
                     'roc_auc_ovr_weighted_test':[roc_auc_b]
        }

    test_results_y_pred_clf_fpipe_b=pd.DataFrame(data=test_results_clf_fpipe_b)
    try:
        os.makedirs(os.path.dirname(path_results_test_b), exist_ok=True)
        test_results_y_pred_clf_fpipe_b.to_excel(path_results_test_b,index=False)
        print(f"Test results 'b' saved to {path_results_test_b}")
    except Exception as e:
        print(f"Error saving test results 'b': {e}")

    print(f"Accuracy of test set (model b): {acc_b:.4f}")
    print(f"F1 (weighted) of test set (model b): {f1_b:.4f}")
    print(f"ROC AUC (OVR weighted) of test set (model b): {roc_auc_b:.4f}" if roc_auc_b is not None else "ROC AUC: N/A (predict_proba not available)")

else:
    print("Model 'b' was not fit successfully. Skipping test phase.")


Evaluating Model B on test set...

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:32 2025 >>>>>>>>Calling transform() from Feature_Selector


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Test results 'b' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_b_multi_random_allocation.xlsx
Accuracy of test set (model b): 0.6772
F1 (weighted) of test set (model b): 0.6492
ROC AUC (OVR weighted) of test set (model b): 0.7532


## **Model C. General approach where feature selection is performed as a whole for other features**

In [44]:
print("\n--- Training Model C (No Feature Selection) ---")
full_parallel_pipe_optc=Pipeline([('data_prep',dataprep_pipe_optc),('clf',DecisionTreeClassifier(random_state=42))]) # Start with default clf
print("Pipeline C keys:")
print(full_parallel_pipe_optc.get_params().keys())


--- Training Model C (No Feature Selection) ---
Pipeline C keys:
dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'data_prep', 'clf', 'data_prep__force_int_remainder_cols', 'data_prep__n_jobs', 'data_prep__remainder', 'data_prep__sparse_threshold', 'data_prep__transformer_weights', 'data_prep__transformers', 'data_prep__verbose', 'data_prep__verbose_feature_names_out', 'data_prep__numeric_pipe', 'data_prep__nominal_pipe', 'data_prep__ordinal_pipe', 'data_prep__numeric_pipe__memory', 'data_prep__numeric_pipe__steps', 'data_prep__numeric_pipe__transform_input', 'data_prep__numeric_pipe__verbose', 'data_prep__numeric_pipe__data_missing', 'data_prep__numeric_pipe__scaler', 'data_prep__numeric_pipe__data_missing__strategy', 'data_prep__numeric_pipe__scaler__clip', 'data_prep__numeric_pipe__scaler__copy', 'data_prep__numeric_pipe__scaler__feature_range', 'data_prep__nominal_pipe__memory', 'data_prep__nominal_pipe__steps', 'data_prep__nominal_pipe__transform_input', 'data_prep__no

In [45]:
#Load the model saved to avoid a new fitting
path_model_c = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/clf_fpipe_c_multi_random_allocation.pkl'
path_results_train_c = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_c_multi_random_allocation.xlsx'
path_results_test_c = r'/content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_c_multi_random_allocation.xlsx'

try:
    clf_fpipe_c= joblib.load(path_model_c)
    print("Loaded model 'c' from disk.")
    model_c_loaded = True
except FileNotFoundError:
    print(f"Model file 'c' not found at {path_model_c}. Fitting new model.")
    clf_fpipe_c = None
    model_c_loaded = False
except Exception as e:
    print(f"Error loading model 'c': {e}. Fitting new model.")
    clf_fpipe_c = None
    model_c_loaded = False

Loaded model 'c' from disk.


In [46]:
# Reduced parameter grid for Option C (only classifier and imputation strategy)
param_grid_fpipe_c={'clf':[dectree_clf, rndforest_clf, extratree_clf, ada_clf, xgboost_clf, gradboost_clf], # Removed voting
                    'data_prep__numeric_pipe__data_missing__strategy':['mean','median']
                    }

In [47]:
if not model_c_loaded or not hasattr(clf_fpipe_c, 'best_estimator_'):
    if model_c_loaded:
        print("Model 'c' loaded but seems unfit. Refitting.")
    #Note: n_iter might be larger than the parameter space for this simple grid.
    #RandomizedSearchCV will warn and run fewer iterations (like GridSearchCV).
    n_iter_c = min(N_ITER_SEARCH, len(param_grid_fpipe_c['clf']) * len(param_grid_fpipe_c['data_prep__numeric_pipe__data_missing__strategy']))
    clf_fpipe_c=RandomizedSearchCV(full_parallel_pipe_optc,
                                 param_distributions=param_grid_fpipe_c,
                                 n_iter=n_iter_c, # Adjust n_iter
                                 scoring=scoring,
                                 refit=refit_metric,
                                 cv=5,
                                 n_jobs=-1, # Use None for debugging if needed, -1 for speed
                                 random_state=42,
                                 error_score='raise',
                                 verbose=1)
    print("Fitting model 'c'...")
    try:
        clf_fpipe_c.fit(X_train, y_train) # Use encoded y_train
        print("Fit complete for model 'c'.")
        #Saving the model
        try:
            os.makedirs(os.path.dirname(path_model_c), exist_ok=True)
            joblib.dump(clf_fpipe_c, path_model_c, compress=1)
            print(f"Model 'c' saved to {path_model_c}")
        except Exception as e:
            print(f"Error saving model 'c': {e}")
    except Exception as e:
         print(f"ERROR during RandomizedSearchCV fit for Model C: {e}")
         import traceback
         traceback.print_exc()
else:
    print("Skipping fit for model 'c' as it was loaded from disk.")

Skipping fit for model 'c' as it was loaded from disk.


In [48]:
#Printing the best estimator for model C
if hasattr(clf_fpipe_c, 'best_params_'):
    print('\nBest estimator of clf_fpipe_c:', clf_fpipe_c.best_params_)
    print(f'Score ({refit_metric}) of best estimator of clf_fpipe_c:', clf_fpipe_c.best_score_)
    print('Best index',clf_fpipe_c.best_index_ )
else:
    print("Model 'c' has not been fitted successfully.")


Best estimator of clf_fpipe_c: {'data_prep__numeric_pipe__data_missing__strategy': 'mean', 'clf': RandomForestClassifier(random_state=42)}
Score (roc_auc_ovr_weighted) of best estimator of clf_fpipe_c: 0.8433699921876705
Best index 2


In [49]:
#Saving the training results into dataframe
if hasattr(clf_fpipe_c, 'cv_results_'):
    df_results_clf_fpipe_c=pd.DataFrame(clf_fpipe_c.cv_results_).sort_values(f'rank_test_{refit_metric}')
    try:
        os.makedirs(os.path.dirname(path_results_train_c), exist_ok=True)
        df_results_clf_fpipe_c.to_excel(path_results_train_c,index=False)
        print(f"Training results 'c' saved to {path_results_train_c}")
    except Exception as e:
        print(f"Error saving training results 'c': {e}")
else:
    print("No CV results to save for model 'c'.")

Training results 'c' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/train_results_clf_fpipe_c_multi_random_allocation.xlsx


In [50]:
#Performing test phase with test set for model C
if hasattr(clf_fpipe_c, 'best_estimator_'):
    print("\nEvaluating Model C on test set...")
    # clf_fpipe_c.refit
    y_pred_clf_fpipe_c=clf_fpipe_c.predict(X_test)
    y_proba_clf_fpipe_c = clf_fpipe_c.predict_proba(X_test) if hasattr(clf_fpipe_c.best_estimator_, 'predict_proba') else None

    acc_c = accuracy_score(y_test, y_pred_clf_fpipe_c)
    f1_c = f1_score(y_test, y_pred_clf_fpipe_c, average='weighted', zero_division=0)
    prec_c = precision_score(y_test, y_pred_clf_fpipe_c, average='weighted', zero_division=0)
    rec_c = recall_score(y_test, y_pred_clf_fpipe_c, average='weighted', zero_division=0)
    mcc_c = matthews_corrcoef(y_test, y_pred_clf_fpipe_c)
    # Correctly call roc_auc_score for multiclass with probabilities
    roc_auc_c = roc_auc_score(y_test, y_proba_clf_fpipe_c, average='weighted', multi_class='ovr') if y_proba_clf_fpipe_c is not None else None

    test_results_clf_fpipe_c={'clf':['clf_fpipe_c_multi_random'],
                     'params':[clf_fpipe_c.best_params_],
                     'accuracy_test':[acc_c],
                     'f1_weighted_test':[f1_c],
                     'precision_weighted_test':[prec_c],
                     'recall_weighted_test':[rec_c],
                     'mcc_test': [mcc_c],
                     'roc_auc_ovr_weighted_test':[roc_auc_c]
        }

    test_results_y_pred_clf_fpipe_c=pd.DataFrame(data=test_results_clf_fpipe_c)
    try:
        os.makedirs(os.path.dirname(path_results_test_c), exist_ok=True)
        test_results_y_pred_clf_fpipe_c.to_excel(path_results_test_c,index=False)
        print(f"Test results 'c' saved to {path_results_test_c}")
    except Exception as e:
        print(f"Error saving test results 'c': {e}")

    print(f"Accuracy of test set (model c): {acc_c:.4f}")
    print(f"F1 (weighted) of test set (model c): {f1_c:.4f}")
    print(f"ROC AUC (OVR weighted) of test set (model c): {roc_auc_c:.4f}" if roc_auc_c is not None else "ROC AUC: N/A (predict_proba not available)")

else:
    print("Model 'c' was not fit successfully. Skipping test phase.")

print("\n--- Multiclass Analysis Complete ---")


Evaluating Model C on test set...

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:23:33 2025 >>>>>>>>Calling transform() from Category_Imputer
Test results 'c' saved to /content/drive/MyDrive/HTIN5005/GridSearchCV_results/test_results_y_pred_clf_fpipe_c_multi_random_allocation.xlsx
Accuracy of test set (model c): 0.7402
F1 (weighted) of test set (model c): 0.7165
ROC AUC (OVR weighted) of test set (model c): 0.8405

--- Multiclass Analysis Complete ---


# **Interpretability Analysis**

In [51]:
#Import self-developed interpretability metrics for analysis
import interpretability_metrics as im

In [52]:
#Determine n (total features before selection)
n_total_features = len(numerical_feats) + len(nominal_feats) + len(ordinal_feats)
print(f"Total features considered for selection (n): {n_total_features}")

Total features considered for selection (n): 16


In [53]:
#Interpretability Metrics for Model A
results_a = {'I': np.nan, 'Acc': np.nan, 'AUROC': np.nan, 'AccII': np.nan, 'AUROCII': np.nan}

if im is not None and hasattr(clf_fpipe_a, 'best_estimator_'):
    print("\n--- Calculating Metrics for Model A (Parallel Feature Selection) ---")
    best_model_a = clf_fpipe_a.best_estimator_
    best_params_a = clf_fpipe_a.best_params_

    # Determine k for Model A
    k_numeric_a = best_params_a.get('data_prep__numeric_pipe__feat_sel__k_out_features', 0)
    k_nominal_a = best_params_a.get('data_prep__nominal_pipe__feat_sel__k_out_features', 0)
    k_ordinal_a = best_params_a.get('data_prep__ordinal_pipe__feat_sel__k_out_features', 0)
    k_a = k_numeric_a + k_nominal_a + k_ordinal_a
    print(f"Features used by Model A (k): {k_a}")

    y_pred_a = best_model_a.predict(X_test)

    # Get scores for AUROCII
    y_scores_a = None
    if hasattr(best_model_a, "predict_proba"):
        y_scores_a = best_model_a.predict_proba(X_test)[:, 1]
    elif hasattr(best_model_a, "decision_function"):
        try:
           y_scores_a = best_model_a.decision_function(X_test)
           if y_scores_a.ndim > 1 and y_scores_a.shape[1] > 1: y_scores_a = y_scores_a[:, 1]
        except Exception as e: print(f"Could not get decision_function scores for Model A: {e}")
    else: print("Model A classifier does not support predict_proba or decision_function.")


    # Calculate metrics
    results_a['I'] = im.calculate_interpretability(k=k_a, n=n_total_features)
    results_a['Acc'] = accuracy_score(y_test, y_pred_a)
    results_a['AccII'] = im.calculate_accii(y_test, y_pred_a, k=k_a, n=n_total_features, alpha=0.5)

    if y_scores_a is not None:
        try:
            results_a['AUROC'] = roc_auc_score(y_test, y_scores_a)
            results_a['AUROCII'] = im.calculate_aurocii(y_test, y_scores_a, k=k_a, n=n_total_features, alpha=0.5)
        except ValueError as e:
            print(f"Could not calculate AUROC/AUROCII for Model A: {e}")
    else:
         print("Cannot calculate AUROC/AUROCII for Model A as scores are unavailable.")

    print(f"Interpretability (I): {results_a['I']:.4f}")
    print(f"Accuracy: {results_a['Acc']:.4f}")
    print(f"AUROC: {results_a['AUROC']:.4f}")
    print(f"AccII (alpha=0.5): {results_a['AccII']:.4f}")
    print(f"AUROCII (alpha=0.5): {results_a['AUROCII']:.4f}")

else:
    print("\nSkipping metrics calculation for Model A (module not loaded or model not fitted).")


--- Calculating Metrics for Model A (Parallel Feature Selection) ---
Features used by Model A (k): 12

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:04 2025 >>>>>>>>Calling transform

/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instanc

In [54]:
#Interpretability Metrics for Model B
results_b = {'I': np.nan, 'Acc': np.nan, 'AUROC': np.nan, 'AccII': np.nan, 'AUROCII': np.nan}

if im is not None and hasattr(clf_fpipe_b, 'best_estimator_'):
    print("\n--- Calculating Metrics for Model B (General Feature Selection) ---")
    best_model_b = clf_fpipe_b.best_estimator_
    best_params_b = clf_fpipe_b.best_params_

    # Determine k for Model B
    k_b = best_params_b.get('data_prep__feat_sel__k_out_features', 0)
    print(f"Features used by Model B (k): {k_b}")

    y_pred_b = best_model_b.predict(X_test)

    # Get scores for AUROCII
    y_scores_b = None
    if hasattr(best_model_b, "predict_proba"):
        y_scores_b = best_model_b.predict_proba(X_test)[:, 1]
    elif hasattr(best_model_b, "decision_function"):
         try:
           y_scores_b = best_model_b.decision_function(X_test)
           if y_scores_b.ndim > 1 and y_scores_b.shape[1] > 1: y_scores_b = y_scores_b[:, 1]
         except Exception as e: print(f"Could not get decision_function scores for Model B: {e}")
    else: print("Model B classifier does not support predict_proba or decision_function.")

    # Calculate metrics
    results_b['I'] = im.calculate_interpretability(k=k_b, n=n_total_features)
    results_b['Acc'] = accuracy_score(y_test, y_pred_b)
    results_b['AccII'] = im.calculate_accii(y_test, y_pred_b, k=k_b, n=n_total_features, alpha=0.5)

    if y_scores_b is not None:
         try:
            results_b['AUROC'] = roc_auc_score(y_test, y_scores_b)
            results_b['AUROCII'] = im.calculate_aurocii(y_test, y_scores_b, k=k_b, n=n_total_features, alpha=0.5)
         except ValueError as e:
            print(f"Could not calculate AUROC/AUROCII for Model B: {e}")

    else:
         print("Cannot calculate AUROC/AUROCII for Model B as scores are unavailable.")

    print(f"Interpretability (I): {results_b['I']:.4f}")
    print(f"Accuracy: {results_b['Acc']:.4f}")
    print(f"AUROC: {results_b['AUROC']:.4f}")
    print(f"AccII (alpha=0.5): {results_b['AccII']:.4f}")
    print(f"AUROCII (alpha=0.5): {results_b['AUROCII']:.4f}")

else:
    print("\nSkipping metrics calculation for Model B (module not loaded or model not fitted).")


--- Calculating Metrics for Model B (General Feature Selection) ---
Features used by Model B (k): 15

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Feature_Selector

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:26 2025 >>>>>>>>Calling transform() from Feature_Selector
Could not calculate AUROC/AUROCII for Model B: multi_class must be in ('ovo', 'ovr')
Interpretability (I): 0.0667
Accuracy: 0.6772
AUROC: nan
AccII (alpha=0.5): 0.3719
AUROCII (alpha=0.5): nan


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [55]:
# Interpretability Metrics for Model C
results_c = {'I': np.nan, 'Acc': np.nan, 'AUROC': np.nan, 'AccII': np.nan, 'AUROCII': np.nan}

if im is not None and hasattr(clf_fpipe_c, 'best_estimator_'):
    print("\n--- Calculating Metrics for Model C (No Feature Selection) ---")
    best_model_c = clf_fpipe_c.best_estimator_
    best_params_c = clf_fpipe_c.best_params_

    # Determine k for Model C (no feature selection means k=n)
    k_c = n_total_features
    print(f"Features used by Model C (k): {k_c} (Same as n)")

    y_pred_c = best_model_c.predict(X_test)

    # Get scores for AUROCII
    y_scores_c = None
    if hasattr(best_model_c, "predict_proba"):
        y_scores_c = best_model_c.predict_proba(X_test)[:, 1]
    elif hasattr(best_model_c, "decision_function"):
         try:
           y_scores_c = best_model_c.decision_function(X_test)
           if y_scores_c.ndim > 1 and y_scores_c.shape[1] > 1: y_scores_c = y_scores_c[:, 1]
         except Exception as e: print(f"Could not get decision_function scores for Model C: {e}")
    else: print("Model C classifier does not support predict_proba or decision_function.")


    # Calculate metrics
    results_c['I'] = im.calculate_interpretability(k=k_c, n=n_total_features)
    results_c['Acc'] = accuracy_score(y_test, y_pred_c)
    results_c['AccII'] = im.calculate_accii(y_test, y_pred_c, k=k_c, n=n_total_features, alpha=0.5)

    if y_scores_c is not None:
        try:
            results_c['AUROC'] = roc_auc_score(y_test, y_scores_c)
            results_c['AUROCII'] = im.calculate_aurocii(y_test, y_scores_c, k=k_c, n=n_total_features, alpha=0.5)
        except ValueError as e:
            print(f"Could not calculate AUROC/AUROCII for Model C: {e}")

    else:
        print("Cannot calculate AUROC/AUROCII for Model C as scores are unavailable.")


    print(f"Interpretability (I): {results_c['I']:.4f}")
    print(f"Accuracy: {results_c['Acc']:.4f}")
    print(f"AUROC: {results_c['AUROC']:.4f}")
    print(f"AccII (alpha=0.5): {results_c['AccII']:.4f}")
    print(f"AUROCII (alpha=0.5): {results_c['AUROCII']:.4f}")
else:
    print("\nSkipping metrics calculation for Model C (module not loaded or model not fitted).")


--- Calculating Metrics for Model C (No Feature Selection) ---
Features used by Model C (k): 16 (Same as n)

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Numeric_Imputer

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Category_Imputer

 Sun Oct 26 07:25:35 2025 >>>>>>>>Calling transform() from Category_Imputer
Could not calculate AUROC/AUROCII for Model C: multi_class must be in ('ovo', 'ovr')
Interpretability (I): 0.0000
Accuracy: 0.7402
AUROC: nan
AccII (alpha=0.5): 0.3701
AUROCII (alpha=0.5): nan


In [56]:
#Summary Table
summary_data = {
    'Model': ['Model A (Parallel FS)', 'Model B (General FS)', 'Model C (No FS)'],
    'k (Features Used)': [k_a if 'k_a' in locals() else 'N/A',
                          k_b if 'k_b' in locals() else 'N/A',
                          k_c if 'k_c' in locals() else 'N/A'],
    'n (Total Features)': [n_total_features] * 3,
    'Interpretability (I)': [results_a['I'], results_b['I'], results_c['I']],
    'Accuracy': [results_a['Acc'], results_b['Acc'], results_c['Acc']],
    'AUROC': [results_a['AUROC'], results_b['AUROC'], results_c['AUROC']],
    'AccII (alpha=0.5)': [results_a['AccII'], results_b['AccII'], results_c['AccII']],
    'AUROCII (alpha=0.5)': [results_a['AUROCII'], results_b['AUROCII'], results_c['AUROCII']]
}

df_summary = pd.DataFrame(summary_data)

print("\n--- Interpretability Metrics Summary (Test Set) ---")
display(df_summary.round(4))


--- Interpretability Metrics Summary (Test Set) ---


,Model,k (Features Used),n (Total Features),Interpretability (I),Accuracy,AUROC,AccII (alpha=0.5),AUROCII (alpha=0.5)
0,Model A (Parallel FS),12,16,0.2667,0.7165,NaN,0.4916,NaN
1,Model B (General FS),15,16,0.0667,0.6772,NaN,0.3719,NaN
2,Model C (No FS),16,16,0.0000,0.7402,NaN,0.3701,NaN
